## DELTA LIVE TABLES - GOLD LAYER

#COACHES DLT PIPELINE

In [0]:
import dlt
from pyspark.sql.types import * 
from pyspark.sql.functions import col

## expectations for data quality

In [0]:
expec_coaches={
        "rule1" : "code is not null",
        "rule2" : "current is True",
        
    }

In [0]:
expec_nocs={
        "rule1" : "code is not null"    
    }

In [0]:
expec_events={
        "rule1" : "event is not null"    
    }

In [0]:
@dlt.table
@dlt.expect_all(expec_coaches)
def source_coaches():
    df = spark.readStream.table("olympics.silver.coaches")
    return df 

In [0]:
@dlt.view
def view_coaches():
    df = spark.readStream.table("LIVE.source_coaches")
    return df

In [0]:
@dlt.table
def coaches():
    df = spark.readStream.table("LIVE.view_coaches")
    return df 

### NOCS dlt pipeline

In [0]:
@dlt.view
def source_nocs():
    df = spark.readStream.table("olympics.silver.nocs")
    return df 

In [0]:
@dlt.table
@dlt.expect_all_or_drop(expec_nocs)
def nocs():
    df = spark.readStream.table("LIVE.source_nocs")
    return df

### Events dlt pipeline 

In [0]:
@dlt.view
def source_events():
    df = spark.readStream.table("olympics.silver.events")
    return df 

In [0]:
@dlt.table
@dlt.expect_all(expec_events)
def events():
    df = spark.readStream.table("LIVE.source_events")
    return df

## CDC apply changes (DLT)

In [0]:
@dlt.table
def source_athletes():
    df = spark.readStream.table("olympics.silver.athletes")
    return df 

In [0]:
dlt.create_streaming_table("athletes")

In [0]:
dlt.apply_changes(
    target = "athletes",
    source = "source_athletes",
    keys = ["athlete_id"],
    sequence_by = col("height"),
    stored_as_scd_type = 1 

)